# Package Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import L2
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
%matplotlib inline

# Preprocessing

In [2]:
DF = pd.read_csv('data_for_ml.csv')
DF.head(2)

,INDEX_,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,3366652,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major
1,3370334,19,weekday,Rain,Dark,Wet,4.8,1.00,Midblock,Traffic Signal including Transit,Major Arterial,60,351.0,63.0,Residential,452,205,183,Over 65,Normal,Crossing without ROW,automobile,speeding,Major


In [3]:
df = DF.drop('INDEX_', axis=1)
df.head(2)

,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major
1,19,weekday,Rain,Dark,Wet,4.8,1.00,Midblock,Traffic Signal including Transit,Major Arterial,60,351.0,63.0,Residential,452,205,183,Over 65,Normal,Crossing without ROW,automobile,speeding,Major


In [4]:
df['INJURY'].isna().sum()

28

In [5]:
df['INJURY'] = df['INJURY'].fillna('None')
df['INJURY'].tail(25)

3088    Major
3089     None
3090    Fatal
3091    Major
3092    Minor
3093    Minor
3094    Major
3095    Minor
3096    Major
3097    Fatal
3098    Major
3099    Major
3100    Major
3101    Major
3102    Fatal
3103    Major
3104    Fatal
3105    Major
3106    Major
3107    Major
3108    Major
3109    Fatal
3110    Major
3111    Major
3112    Fatal
Name: INJURY, dtype: object

In [6]:
mapping = {
    'Fatal' : 4,
    'Major' : 3,
    'Minor' : 2,
    'Minimal' : 1,
    'None' : 0
}
df['INJURY'] = df['INJURY'].replace(mapping).astype(int)
df['INJURY'].head(6)

0    3
1    3
2    3
3    4
4    4
5    3
Name: INJURY, dtype: int32

In [7]:
# Split into X and y variables
X = df.iloc[:, :-1].values
y = df.iloc[:,-1].values

In [8]:
# One hot encode the categorical columns
cat_columns_index = [1, 2, 3, 4, 7, 8, 9, 13, 17, 18, 19, 20, 21]
encoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X[:, cat_columns_index])

In [9]:
# Combine the categorical and numerical columns
num_columns_index = [0, 5, 6, 10, 11, 12, 14, 15, 16]
X_encoded = np.concatenate((X_cat, X[:, num_columns_index].astype(float)), axis=1)
print(X_encoded)

[[1.000e+00 0.000e+00 0.000e+00 ... 5.040e+02 2.630e+02 2.250e+02]
 [1.000e+00 0.000e+00 0.000e+00 ... 4.520e+02 2.050e+02 1.830e+02]
 [1.000e+00 0.000e+00 1.000e+00 ... 8.070e+02 3.750e+02 3.220e+02]
 ...
 [1.000e+00 0.000e+00 0.000e+00 ... 4.820e+02 2.520e+02 2.310e+02]
 [0.000e+00 1.000e+00 1.000e+00 ... 1.981e+03 1.195e+03 1.112e+03]
 [1.000e+00 0.000e+00 1.000e+00 ... 1.170e+03 4.820e+02 4.670e+02]]


In [10]:
# Split encoded X and y to train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.20, random_state=100)

In [11]:
# Scale the numerical features
scaler = StandardScaler()
X_train[:, -len(num_columns_index):] = scaler.fit_transform(X_train[:, -len(num_columns_index):])
X_test[:, -len(num_columns_index):] = scaler.transform(X_test[:, -len(num_columns_index):])

# ANN Modeling

## 1 Shallow Network

In [12]:
# Define architecture
input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
hidden_layer = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_shallow = Sequential([input_layer, hidden_layer, output_layer])
# Compile
model_shallow.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [13]:
model_shallow.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 1s 1ms/step - loss: 1.4841 - accuracy: 0.7719
Epoch 2/200
78/78 [==============================] - 0s 1ms/step - loss: 1.1082 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 1ms/step - loss: 0.9601 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 1ms/step - loss: 0.8759 - accuracy: 0.7755
Epoch 5/200
78/78 [==============================] - 0s 1ms/step - loss: 0.8240 - accuracy: 0.7767
Epoch 6/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7954 - accuracy: 0.7787
Epoch 7/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7737 - accuracy: 0.7807
Epoch 8/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7649 - accuracy: 0.7807
Epoch 9/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7493 - accuracy: 0.7811
Epoch 10/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7414 - accuracy: 0.7803
Epoch 11/

78/78 [==============================] - 0s 1ms/step - loss: 0.6987 - accuracy: 0.7843
Epoch 84/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6996 - accuracy: 0.7859
Epoch 85/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6993 - accuracy: 0.7799
Epoch 86/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6991 - accuracy: 0.7839
Epoch 87/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6981 - accuracy: 0.7807
Epoch 88/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6995 - accuracy: 0.7811
Epoch 89/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6983 - accuracy: 0.7807
Epoch 90/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6998 - accuracy: 0.7823
Epoch 91/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6982 - accuracy: 0.7823
Epoch 92/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6998 - accuracy: 0.7827
Epoch 93/200


78/78 [==============================] - 0s 1ms/step - loss: 0.6963 - accuracy: 0.7819
Epoch 165/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6939 - accuracy: 0.7831
Epoch 166/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6947 - accuracy: 0.7839
Epoch 167/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6940 - accuracy: 0.7831
Epoch 168/200
78/78 [==============================] - 0s 2ms/step - loss: 0.6961 - accuracy: 0.7783
Epoch 169/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6942 - accuracy: 0.7823
Epoch 170/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6946 - accuracy: 0.7839
Epoch 171/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6942 - accuracy: 0.7811
Epoch 172/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6943 - accuracy: 0.7815
Epoch 173/200
78/78 [==============================] - 0s 1ms/step - loss: 0.6946 - accuracy: 0.7827
Epoc

In [14]:
loss_test, accuracy_test = model_shallow.evaluate(X_test, y_test)

20/20 [==============================] - 0s 1ms/step - loss: 0.7263 - accuracy: 0.7785


In [15]:
loss_train, accuracy_train = model_shallow.evaluate(X_train, y_train)

78/78 [==============================] - 0s 1ms/step - loss: 0.6914 - accuracy: 0.7811


## 2 Medium Network

In [16]:
# Define architecture
m_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
m_hidden_layer_1 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
m_hidden_layer_2 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
m_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_medium = Sequential([m_output_layer, m_hidden_layer_1, m_hidden_layer_2, m_output_layer])
# Compile
model_medium.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [17]:
model_medium.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 1s 1ms/step - loss: 3.4237 - accuracy: 0.6936
Epoch 2/200
78/78 [==============================] - 0s 1ms/step - loss: 1.1328 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 1ms/step - loss: 0.9589 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 1ms/step - loss: 0.9152 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 1ms/step - loss: 0.8903 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 1ms/step - loss: 0.8765 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 1ms/step - loss: 0.8593 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 1ms/step - loss: 0.8475 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8390 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8303 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 1ms/step - loss: 0.7284 - accuracy: 0.7839
Epoch 84/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7274 - accuracy: 0.7811
Epoch 85/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7260 - accuracy: 0.7831
Epoch 86/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7294 - accuracy: 0.7855
Epoch 87/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7264 - accuracy: 0.7819
Epoch 88/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7252 - accuracy: 0.7811
Epoch 89/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7238 - accuracy: 0.7831
Epoch 90/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7246 - accuracy: 0.7807
Epoch 91/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7233 - accuracy: 0.7807
Epoch 92/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7245 - accuracy: 0.7807
Epoch 93/200


78/78 [==============================] - 0s 1ms/step - loss: 0.7082 - accuracy: 0.7807
Epoch 165/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7087 - accuracy: 0.7823
Epoch 166/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7107 - accuracy: 0.7839
Epoch 167/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7111 - accuracy: 0.7831
Epoch 168/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7139 - accuracy: 0.7823
Epoch 169/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7090 - accuracy: 0.7823
Epoch 170/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7113 - accuracy: 0.7811
Epoch 171/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7102 - accuracy: 0.7823
Epoch 172/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7060 - accuracy: 0.7831
Epoch 173/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7135 - accuracy: 0.7791
Epoc

In [18]:
m_loss_test, m_accuracy_test = model_medium.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.7404 - accuracy: 0.7721


In [19]:
m_loss_train, m_accuracy_train = model_medium.evaluate(X_train, y_train)

78/78 [==============================] - 0s 1ms/step - loss: 0.7068 - accuracy: 0.7831


## 3 Deep Network Version 1

In [20]:
# Define architecture
deep1_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
deep1_hidden_layer_1 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep1_hidden_layer_2 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep1_hidden_layer_3 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep1_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_deep1 = Sequential([deep1_input_layer,
                         deep1_hidden_layer_1,
                         deep1_hidden_layer_2,
                         deep1_hidden_layer_3,
                         deep1_output_layer])
# Compile
model_deep1.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [21]:
model_deep1.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 1s 2ms/step - loss: 2.6095 - accuracy: 0.7353
Epoch 2/200
78/78 [==============================] - 0s 2ms/step - loss: 1.4594 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 2ms/step - loss: 1.0839 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 2ms/step - loss: 0.9336 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8661 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8275 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8059 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7938 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7821 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7728 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 2ms/step - loss: 0.7195 - accuracy: 0.7843
Epoch 84/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7191 - accuracy: 0.7791
Epoch 85/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7198 - accuracy: 0.7815
Epoch 86/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7191 - accuracy: 0.7839
Epoch 87/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7203 - accuracy: 0.7831
Epoch 88/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7213 - accuracy: 0.7847
Epoch 89/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7201 - accuracy: 0.7811
Epoch 90/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7199 - accuracy: 0.7823
Epoch 91/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7185 - accuracy: 0.7843
Epoch 92/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7197 - accuracy: 0.7831
Epoch 93/200


78/78 [==============================] - 0s 2ms/step - loss: 0.7136 - accuracy: 0.7835
Epoch 165/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7133 - accuracy: 0.7843
Epoch 166/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7128 - accuracy: 0.7803
Epoch 167/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7167 - accuracy: 0.7819
Epoch 168/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7141 - accuracy: 0.7843
Epoch 169/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7129 - accuracy: 0.7835
Epoch 170/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7130 - accuracy: 0.7847
Epoch 171/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7136 - accuracy: 0.7831
Epoch 172/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7142 - accuracy: 0.7823
Epoch 173/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7148 - accuracy: 0.7855
Epoc

In [22]:
d1_loss_test, d1_accuracy_test = model_deep1.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.7437 - accuracy: 0.7737


In [23]:
d1_loss_train, d1_accuracy_train = model_deep1.evaluate(X_train, y_train)

78/78 [==============================] - 0s 1ms/step - loss: 0.7082 - accuracy: 0.7843


## 4 Deep Network Version 2

In [24]:
# Define architecture
deep2_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
deep2_hidden_layer_1 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep2_hidden_layer_2 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep2_hidden_layer_3 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep2_hidden_layer_4 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep2_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_deep2 = Sequential([deep2_input_layer,
                         deep2_hidden_layer_1,
                         deep2_hidden_layer_2,
                         deep2_hidden_layer_3,
                         deep2_hidden_layer_4,
                         deep2_output_layer])
# Compile
model_deep2.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [25]:
model_deep2.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 1s 2ms/step - loss: 3.2346 - accuracy: 0.7643
Epoch 2/200
78/78 [==============================] - 0s 2ms/step - loss: 1.6125 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 2ms/step - loss: 1.1279 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 2ms/step - loss: 0.9584 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8839 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8452 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8172 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8037 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7903 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7834 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 1ms/step - loss: 0.7331 - accuracy: 0.7803
Epoch 84/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7291 - accuracy: 0.7835
Epoch 85/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7301 - accuracy: 0.7827
Epoch 86/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7288 - accuracy: 0.7831
Epoch 87/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7281 - accuracy: 0.7815
Epoch 88/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7292 - accuracy: 0.7831
Epoch 89/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7298 - accuracy: 0.7851
Epoch 90/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7279 - accuracy: 0.7831
Epoch 91/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7319 - accuracy: 0.7827
Epoch 92/200
78/78 [==============================] - 0s 1ms/step - loss: 0.7291 - accuracy: 0.7795
Epoch 93/200


78/78 [==============================] - 0s 2ms/step - loss: 0.7244 - accuracy: 0.7843
Epoch 165/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7235 - accuracy: 0.7827
Epoch 166/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7235 - accuracy: 0.7827
Epoch 167/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7220 - accuracy: 0.7835
Epoch 168/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7235 - accuracy: 0.7831
Epoch 169/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7238 - accuracy: 0.7831
Epoch 170/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7230 - accuracy: 0.7803
Epoch 171/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7224 - accuracy: 0.7811
Epoch 172/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7244 - accuracy: 0.7827
Epoch 173/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7246 - accuracy: 0.7811
Epoc

In [26]:
d2_loss_test, d2_accuracy_test = model_deep2.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.7534 - accuracy: 0.7801


In [27]:
d2_loss_train, d2_accuracy_train = model_deep2.evaluate(X_train, y_train)

78/78 [==============================] - 0s 2ms/step - loss: 0.7202 - accuracy: 0.7819


## 5 Deep Network Version 3

In [28]:
# Define architecture
deep3_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
deep3_hidden_layer_1 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep3_hidden_layer_2 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep3_hidden_layer_3 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep3_hidden_layer_4 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep3_hidden_layer_5 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep3_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_deep3 = Sequential([deep3_input_layer,
                         deep3_hidden_layer_1,
                         deep3_hidden_layer_2,
                         deep3_hidden_layer_3,
                         deep3_hidden_layer_4,
                         deep3_hidden_layer_5,
                         deep3_output_layer])
# Compile
model_deep3.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [29]:
model_deep3.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 2s 3ms/step - loss: 4.0511 - accuracy: 0.7695
Epoch 2/200
78/78 [==============================] - 0s 3ms/step - loss: 1.6277 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 3ms/step - loss: 1.0924 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 2ms/step - loss: 0.9420 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8712 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8397 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8182 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8042 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7974 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7883 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 2ms/step - loss: 0.7248 - accuracy: 0.7751
Epoch 84/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7253 - accuracy: 0.7751
Epoch 85/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7252 - accuracy: 0.7751
Epoch 86/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7255 - accuracy: 0.7751
Epoch 87/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7239 - accuracy: 0.7751
Epoch 88/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7249 - accuracy: 0.7751
Epoch 89/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7255 - accuracy: 0.7751
Epoch 90/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7238 - accuracy: 0.7751
Epoch 91/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7239 - accuracy: 0.7751
Epoch 92/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7232 - accuracy: 0.7751
Epoch 93/200


78/78 [==============================] - 0s 2ms/step - loss: 0.7208 - accuracy: 0.7751
Epoch 165/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7210 - accuracy: 0.7751
Epoch 166/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7210 - accuracy: 0.7751
Epoch 167/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7216 - accuracy: 0.7751
Epoch 168/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7211 - accuracy: 0.7751
Epoch 169/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7215 - accuracy: 0.7751
Epoch 170/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7202 - accuracy: 0.7751
Epoch 171/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7205 - accuracy: 0.7751
Epoch 172/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7208 - accuracy: 0.7751
Epoch 173/200
78/78 [==============================] - 0s 2ms/step - loss: 0.7202 - accuracy: 0.7751
Epoc

In [30]:
d3_loss_test, d3_accuracy_test = model_deep3.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.7234 - accuracy: 0.7833


In [31]:
d3_loss_train, d3_accuracy_train = model_deep3.evaluate(X_train, y_train)

78/78 [==============================] - 0s 2ms/step - loss: 0.7190 - accuracy: 0.7751


## 6 Deep Network Version 4

In [32]:
# Define architecture
deep4_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
deep4_hidden_layer_1 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep4_hidden_layer_2 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep4_hidden_layer_3 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep4_hidden_layer_4 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep4_hidden_layer_5 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep4_hidden_layer_6 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep4_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_deep4 = Sequential([deep4_input_layer,
                         deep4_hidden_layer_1,
                         deep4_hidden_layer_2,
                         deep4_hidden_layer_3,
                         deep4_hidden_layer_4,
                         deep4_hidden_layer_5,
                         deep4_hidden_layer_6,
                         deep4_output_layer])
# Compile
model_deep4.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [33]:
model_deep4.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 2s 3ms/step - loss: 5.1410 - accuracy: 0.7647
Epoch 2/200
78/78 [==============================] - 0s 3ms/step - loss: 1.7170 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 3ms/step - loss: 1.1132 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 3ms/step - loss: 0.9529 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8818 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 4ms/step - loss: 0.8498 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8290 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8162 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8071 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 2ms/step - loss: 0.8015 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 3ms/step - loss: 0.7257 - accuracy: 0.7751
Epoch 84/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7260 - accuracy: 0.7751
Epoch 85/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7246 - accuracy: 0.7751
Epoch 86/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7253 - accuracy: 0.7751
Epoch 87/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7244 - accuracy: 0.7751
Epoch 88/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7251 - accuracy: 0.7751
Epoch 89/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7265 - accuracy: 0.7751
Epoch 90/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7241 - accuracy: 0.7751
Epoch 91/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7253 - accuracy: 0.7751
Epoch 92/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7241 - accuracy: 0.7751
Epoch 93/200


78/78 [==============================] - 0s 3ms/step - loss: 0.7210 - accuracy: 0.7751
Epoch 165/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7222 - accuracy: 0.7751
Epoch 166/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7219 - accuracy: 0.7751
Epoch 167/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7211 - accuracy: 0.7751
Epoch 168/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7216 - accuracy: 0.7751
Epoch 169/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7213 - accuracy: 0.7751
Epoch 170/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7210 - accuracy: 0.7751
Epoch 171/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7203 - accuracy: 0.7751
Epoch 172/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7206 - accuracy: 0.7751
Epoch 173/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7217 - accuracy: 0.7751
Epoc

In [34]:
d4_loss_test, d4_accuracy_test = model_deep4.evaluate(X_test, y_test)

20/20 [==============================] - 0s 3ms/step - loss: 0.7237 - accuracy: 0.7833


In [35]:
d4_loss_train, d4_accuracy_train = model_deep4.evaluate(X_train, y_train)

78/78 [==============================] - 0s 2ms/step - loss: 0.7193 - accuracy: 0.7751


## 7 Deep Network Version 5

In [37]:
# Define architecture
deep5_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
deep5_hidden_layer_1 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep5_hidden_layer_2 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep5_hidden_layer_3 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep5_hidden_layer_4 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep5_hidden_layer_5 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep5_hidden_layer_6 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep5_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_deep5 = Sequential([deep5_input_layer,
                         deep5_hidden_layer_1,
                         deep5_hidden_layer_2,
                         deep5_hidden_layer_3,
                         deep5_hidden_layer_4,
                         deep5_hidden_layer_5,
                         deep5_hidden_layer_6,
                         deep5_output_layer])
# Compile
model_deep5.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [38]:
model_deep5.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 1s 4ms/step - loss: 5.6428 - accuracy: 0.7663
Epoch 2/200
78/78 [==============================] - 0s 3ms/step - loss: 1.6002 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 3ms/step - loss: 1.0617 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 3ms/step - loss: 0.9310 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8706 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8364 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8205 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8082 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8051 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7992 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 3ms/step - loss: 0.7247 - accuracy: 0.7751
Epoch 84/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7248 - accuracy: 0.7751
Epoch 85/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7245 - accuracy: 0.7751
Epoch 86/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7249 - accuracy: 0.7751
Epoch 87/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7240 - accuracy: 0.7751
Epoch 88/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7240 - accuracy: 0.7751
Epoch 89/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7241 - accuracy: 0.7751
Epoch 90/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7231 - accuracy: 0.7751
Epoch 91/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7243 - accuracy: 0.7751
Epoch 92/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7259 - accuracy: 0.7751
Epoch 93/200


78/78 [==============================] - 0s 3ms/step - loss: 0.7213 - accuracy: 0.7751
Epoch 165/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7228 - accuracy: 0.7751
Epoch 166/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7205 - accuracy: 0.7751
Epoch 167/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7208 - accuracy: 0.7751
Epoch 168/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7215 - accuracy: 0.7751
Epoch 169/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7211 - accuracy: 0.7751
Epoch 170/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7207 - accuracy: 0.7751
Epoch 171/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7213 - accuracy: 0.7751
Epoch 172/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7218 - accuracy: 0.7751
Epoch 173/200
78/78 [==============================] - 0s 3ms/step - loss: 0.7206 - accuracy: 0.7751
Epoc

In [39]:
d5_loss_test, d5_accuracy_test = model_deep5.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.7226 - accuracy: 0.7833


In [40]:
d5_loss_train, d5_accuracy_train = model_deep5.evaluate(X_train, y_train)

78/78 [==============================] - 0s 2ms/step - loss: 0.7189 - accuracy: 0.7751


## 8 Deep Network Version 6

In [41]:
# Define architecture
deep6_input_layer = Dense(units=22, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_1 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_2 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_3 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_4 = Dense(units=512, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_5 = Dense(units=256, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_6 = Dense(units=128, activation='relu', kernel_regularizer=L2(0.01))
deep6_hidden_layer_7 = Dense(units=64, activation='relu', kernel_regularizer=L2(0.01))
deep6_output_layer = Dense(units=5, activation='linear', kernel_regularizer=L2(0.01))

model_deep6 = Sequential([deep6_input_layer,
                         deep6_hidden_layer_1,
                         deep6_hidden_layer_2,
                         deep6_hidden_layer_3,
                         deep6_hidden_layer_4,
                         deep6_hidden_layer_5,
                         deep6_hidden_layer_6,
                         deep6_hidden_layer_7,
                         deep6_output_layer])
# Compile
model_deep6.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [42]:
model_deep6.fit(X_train, y_train, epochs=200)

Epoch 1/200
78/78 [==============================] - 2s 4ms/step - loss: 6.3283 - accuracy: 0.7651
Epoch 2/200
78/78 [==============================] - 0s 4ms/step - loss: 1.7167 - accuracy: 0.7751
Epoch 3/200
78/78 [==============================] - 0s 4ms/step - loss: 1.1385 - accuracy: 0.7751
Epoch 4/200
78/78 [==============================] - 0s 3ms/step - loss: 0.9692 - accuracy: 0.7751
Epoch 5/200
78/78 [==============================] - 0s 4ms/step - loss: 0.8981 - accuracy: 0.7751
Epoch 6/200
78/78 [==============================] - 0s 4ms/step - loss: 0.8608 - accuracy: 0.7751
Epoch 7/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8471 - accuracy: 0.7751
Epoch 8/200
78/78 [==============================] - 0s 4ms/step - loss: 0.8252 - accuracy: 0.7751
Epoch 9/200
78/78 [==============================] - 0s 4ms/step - loss: 0.8118 - accuracy: 0.7751
Epoch 10/200
78/78 [==============================] - 0s 3ms/step - loss: 0.8022 - accuracy: 0.7751
Epoch 11/

78/78 [==============================] - 0s 5ms/step - loss: 0.7254 - accuracy: 0.7751
Epoch 84/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7243 - accuracy: 0.7751
Epoch 85/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7245 - accuracy: 0.7751
Epoch 86/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7245 - accuracy: 0.7751
Epoch 87/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7249 - accuracy: 0.7751
Epoch 88/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7242 - accuracy: 0.7751
Epoch 89/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7239 - accuracy: 0.7751
Epoch 90/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7247 - accuracy: 0.7751
Epoch 91/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7248 - accuracy: 0.7751
Epoch 92/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7245 - accuracy: 0.7751
Epoch 93/200


78/78 [==============================] - 0s 5ms/step - loss: 0.7217 - accuracy: 0.7751
Epoch 165/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7209 - accuracy: 0.7751
Epoch 166/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7210 - accuracy: 0.7751
Epoch 167/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7207 - accuracy: 0.7751
Epoch 168/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7208 - accuracy: 0.7751
Epoch 169/200
78/78 [==============================] - 0s 5ms/step - loss: 0.7215 - accuracy: 0.7751
Epoch 170/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7204 - accuracy: 0.7751
Epoch 171/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7196 - accuracy: 0.7751
Epoch 172/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7213 - accuracy: 0.7751
Epoch 173/200
78/78 [==============================] - 0s 4ms/step - loss: 0.7205 - accuracy: 0.7751
Epoc

In [43]:
d6_loss_test, d6_accuracy_test = model_deep6.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.7236 - accuracy: 0.7833


In [44]:
d6_loss_train, d6_accuracy_train = model_deep6.evaluate(X_train, y_train)

78/78 [==============================] - 0s 2ms/step - loss: 0.7190 - accuracy: 0.7751
